In [3]:
using ExpFamilyPCA

n_samples = 5
n_dims = 20
X = rand(n_samples, n_dims)  # generate fake probability data
X ./= sum(X, dims=2)

n_components = 2
epca = PoissonPCA(n_components, n_dims)
fit!(epca, X; verbose=true, maxiter=20)

X̃1 = compress(epca, X)
recon1 = decompress(epca, X̃1)

150.29052838654067
66.85825799955668


6.629823358086027
0.9738082489113375
0.8620017281501285
0.7612694327154735
0.7070667501057029


0.6752490168810463
0.6566693343824469
0.6449862420360407
0.6297493827166457
0.6136629241684936


0.6045034711663557
0.5997138829313783
0.5972248685733998
0.5951450454643485
0.593609798000764


0.5915065117633705
0.5897011150841652
0.5885828692618534


5×20 Matrix{Float64}:
 0.0107295  0.00515195  0.0509353  …  0.041509   0.07893    0.0273867
 0.0856614  0.0426146   0.079095      0.0227859  0.0362736  0.11462
 0.0184289  0.00908704  0.0602435     0.0399778  0.0720944  0.0402574
 0.0825272  0.0411829   0.0793789     0.0236289  0.037681   0.112014
 0.0664109  0.033458    0.0789556     0.0275444  0.0445196  0.0973623

In [4]:
using MultivariateStats

M = fit(PCA, X; maxoutdim=n_components)
X̃2 = predict(M, X)
recon2 = reconstruct(M, X̃2)

5×20 Matrix{Float64}:
 0.0160277  0.037667   0.0284812  …  0.0650312   0.0786847   0.00674484
 0.0863725  0.059657   0.0752105     0.0339471   0.0104532   0.0964759
 0.0311483  0.0282345  0.0471927     0.0585103   0.0287506   0.0266902
 0.0835628  0.0842782  0.0577355     0.0348994   0.0766921   0.0917077
 0.0713507  0.041425   0.0735173     0.0407385  -0.00869196  0.0779429

In [14]:
using LinearAlgebra
using Statistics

dist1 = X .- recon1
dist2 = X .- recon2

KL_divergence(p, q) = sum(p .* log.(p ./ q))
KL1 = mean([KL_divergence(p, q) for (p, q) in zip(eachrow(X), eachrow(recon1))])
KL2 = mean([KL_divergence(p, q) for (p, q) in zip(eachrow(X), eachrow(recon2))])

# From theory, we expect the EPCA reconstruction to do better on the L1 norm but worse on L2
println("EPCA L2:", norm(dist1, 2))
println("PCA L2:", norm(dist2, 2))
println()
println("EPCA KL:", KL1)
println("PCA KL:", KL2)

DomainError: DomainError with -3.7748908205601475:
log was called with a negative real argument but will only return a complex result if called with a complex argument. Try log(Complex(x)).
DomainError detected in the user `f` function. This occurs when the domain of a function is violated.
For example, `log(-1.0)` is undefined because `log` of a real number is defined to only output real
numbers, but `log` of a negative number is complex valued and therefore Julia throws a DomainError
by default. Cases to be aware of include:

* `log(x)`, `sqrt(x)`, `cbrt(x)`, etc. where `x<0`
* `x^y` for `x<0` floating point `y` (example: `(-1.0)^(1/2) == im`)

Within the context of SciML, this error can occur within the solver process even if the domain constraint
would not be violated in the solution due to adaptivity. For example, an ODE solver or optimization
routine may check a step at `new_u` which violates the domain constraint, and if violated reject the
step and use a smaller `dt`. However, 